In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_autophagy as cra
importlib.reload(cra)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pyvista as pv
import multiprocessing as mp
from pathlib import Path
import os

output_path = cra.get_last_output_path()
# output_path = Path("out/autophagy/2023-11-24-18-05-41")
print(output_path)
simulation_settings = cra.get_simulation_settings(output_path)
iter_0_particles = cra.get_particles_at_iter(output_path, 0)
for col in iter_0_particles.columns:
    print(col)
max_iter = max(cra.get_all_iterations(output_path))

In [ ]:
data_cells = cra.get_particles_at_iter(output_path, max_iter)
cargo_pos = np.array([x for x in data_cells[data_cells["element.cell.interaction.species"]=="Cargo"]["element.cell.mechanics.pos"]])
atg11_pos = np.array([x for x in data_cells[data_cells["element.cell.interaction.species"]!="Cargo"]["element.cell.mechanics.pos"]])
cargo_middle = np.average(cargo_pos, axis=0)

dist1 = np.sqrt(np.sum((cargo_pos - cargo_middle)**2, axis=1))
dist2 = np.sqrt(np.sum((atg11_pos - cargo_middle)**2, axis=1))

fig, ax = plt.subplots()
bins = np.histogram(np.hstack((dist1, dist2)), bins=30)[1]
ax.hist(dist1, label="Cargo Particles", bins=bins)
ax.hist(dist2, label="R11 Particles", bins=bins)
ax.set_ylabel("Number of Particles")
ax.set_xlabel("Distance to Cargo center")
ax.legend()
plt.show(fig)

In [ ]:
import scipy as sp
data_cells = cra.get_particles_at_iter(output_path, max_iter)

cargo_mask = data_cells["element.cell.interaction.species"]=="Cargo"
data = np.array([x for x in data_cells["element.cell.mechanics.pos"]])
whitened = sp.cluster.vq.whiten(data)
whitened_cargo = sp.cluster.vq.whiten(data)[cargo_mask]
whitened_atg11 = sp.cluster.vq.whiten(data)[cargo_mask==False]
book_atg11 = np.array((whitened_atg11[0], whitened_atg11[2]))
codebook_cargo, distortion_cargo = sp.cluster.vq.kmeans(whitened_cargo, 1)
codebook_atg11, distoration_atg11 = sp.cluster.vq.kmeans(whitened_atg11, book_atg11)

fig, ax = plt.subplots(figsize=(8, 8))
# Plot the points of particles
ax.scatter(whitened_cargo[:,2], whitened_cargo[:,1], color="#6172ed")
ax.scatter(whitened[cargo_mask==False][:,2], whitened[cargo_mask==False][:,1], color="#e6cb8a")

# Plot the located clusters
ax.scatter(codebook_cargo[:,2], codebook_cargo[:,1], color="#000b5c")
ax.scatter(codebook_atg11[:,2], codebook_atg11[:,1], color="#ff7700")

plt.show(fig)

In [ ]:
# Save all snapshots
cra.save_all_snapshots(output_path, threads=12)

# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/snapshots/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/snapshot_movie.mp4"
os.system(bashcmd)